In [ ]:
# CIFAR-10 dataset
# The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
# The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

# 레이블 : airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck

import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input,Flatten,Dense
from keras.models import Sequential,Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.datasets import cifar10


In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)  # (50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)

plt.figure(figsize=(10, 3))

plt.subplot(1, 3, 1)
plt.imshow(x_train[0])
plt.title(f'Label: {y_train[0]}')

plt.subplot(1, 3, 2)
plt.imshow(x_train[1])
plt.title(f'Label: {y_train[1]}')

plt.subplot(1, 3, 3)
plt.imshow(x_train[2])
plt.title(f'Label: {y_train[2]}')

plt.tight_layout()
plt.show()


In [ ]:
# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

NUM_CLASSES = 10
# Convert labels to categorical format
y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

print(x_train[0, 12, 13, 1])  # Accessing pixel value of the first image: 0번 채널 12행 13열 1번 채널 (그린 채널)


In [ ]:
# CNN 없이 모델 작성

# Sequential API 사용
# model = Sequential([
#     Dense(units=256, activation='relu', input_shape=(32,32,3)),
#     Flatten(),
#     Dense(units=128, activation='relu'),
#     Dense(units=NUM_CLASSES, activation='softmax'),
# ])
# print(model.summary())

# Functional API 사용
# input_layer = Input((32, 32, 3))
# net = Flatten()(input_layer)
# net = Dense(units=256, activation='relu')(net)
# output_layer = Dense(units=NUM_CLASSES, activation='softmax')(net)
# model = Model(input_layer, output_layer)
# print(model.summary())


# CNN 레이어 Dense위에 추가
from keras.layers import Conv2D, MaxPool2D, Activation, ReLU, LeakyReLU, BatchNormalization
# BatchNormalization : 활성화 함수의 값이나 출력값을 정규화함. 학습속도 개선, 가중치 초기값 선택 의존성이 적어짐 => 과적합 방지
input_layer = Input((32, 32, 3))
net = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(input_layer)
net = BatchNormalization()(net)
net = LeakyReLU()(net)
net = MaxPool2D(pool_size=(2,2))(net)  # dropout 말고 batchnormalization쓰는 법

net = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(net)
net = BatchNormalization()(net)
net = LeakyReLU()(net)
net = MaxPool2D(pool_size=(2,2))(net)

net = Flatten()(net)

net = Dense(units=256, activation='relu')(net)
net = BatchNormalization()(net)
net = LeakyReLU()(net)

net = Dense(units=128)(net)
net = BatchNormalization()(net)
net = LeakyReLU()(net)

output_layer = Dense(units=NUM_CLASSES, activation='softmax')(net)
model = Model(input_layer, output_layer)
print(model.summary())

In [ ]:
# 모델 컴파일
opt = Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(x_train, y_train, batch_size=128, epochs=10, shuffle=True, verbose=2)

# 모델 평가
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0, batch_size=128)
print('테스트 정확도:', test_accuracy)  # 테스트 정확도 출력
print('테스트 손실:', test_loss)  # 테스트 손실 출력

# 클래스 레이블 매핑
CLASSES = np.array(['비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말', '배', '트럭'])

In [ ]:
#예측
pred= model.predict(x_test[:10])
pred_single=CLASSES[np.argmax(pred,axis=1)]
actual_single=CLASSES[np.argmax(y_test[:10],axis=1)]
print('예측값 : ', pred_single)
print('실제값 : ', actual_single)
print('분류 실패 수 : ', (pred_single!=actual_single).sum())

In [ ]:


#시각화
fig= plt.figure(figsize=(15,3))
fig.subplots_adjust(hspace=0.4, wspace=0.2)

for i, idx in enumerate(range(len(x_test[:10]))):
    img= x_test[idx]
    ax= fig.add_subplot(1,len(x_test[:10]),i+1)
    ax.axis('off')
    ax.text(0.5,-0.3,'pred='+str(pred_single[idx]), fontdict={'fontsize':10,'color':'blue'}, ha='center', transform=ax.transAxes)
    ax.text(0.5,-0.7,'actu='+str(actual_single[idx]), fontdict={'fontsize':10}, ha='center', transform=ax.transAxes)
    ax.imshow(img)
plt.show()